# Mi primera red neuronal basada en JAX


Para definir y entrenar redes neuronales artificiales con JAX utilizaremos dos librerías:

[Flax](https://flax.readthedocs.io/en/latest/) es una librería de redes neuronales artificiales cuyo foco es la flexibilidad y que utiliza JAX como backend de cómputo, Además tiene interoperabilidad con la librería de programación probabilística *NumPyro*. 

> Flax provee `flax.linen`, una API con primitivas para diseñar redes neuronales (similar a `torch.nn`)

[optax](https://github.com/deepmind/optax) es una librería de optimización numérica para modelos paramétricos basada en JAX. Esta librería provee los algoritmos basados en gradiente descedente que se ocupan típicamente para entrenar redes neuronales artificiales.

## Instalación

Instala utilizando pip:

    pip install flax
    pip install optax


In [ ]:
import flax.linen as nn
import optax

## Definiendo un modelo en `flax`

Un modelo en `flax` es una clase de Python que hereda de `flax.linen.Module`. Existen dos formas de escribir un modelo: explicita o *inline*

En la forma explícitala clase que representa el modelo debe implementar:

- Un método `__call__` que recibe los datos de entrada y retorna una predicción
- Un método `__setup__` que declara las variables y submódulos que componen el modelo 

En la forma *inline* sólo se define `__call__` con un decorador `nn.compact`. Por ejemplo un regresor logístico 

$$
y = \text{sigmoid}\left(\sum_j w_j x_j + b \right)
$$

se implementaría como:

In [4]:
class LogisticRegressor(nn.Module):
    
    @nn.compact
    def __call__(self, x):
        return nn.sigmoid(nn.Dense(1)(x))
    
LogisticRegressor()

LogisticRegressor()

> El decorador se hace cargo de registrar la llamada a los módulos como `nn.Dense`. 

Implementemos ahora el siguiente modelo tipo *multi layer perceptron* con una capa oculta:

$$
y = \left(\sum_j w_j \text{ReLU}\left( \sum_i w_ij x_i + b_i\right) + b \right)
$$


In [12]:
class MLP_singlehidden(nn.Module):
    
    hidden_neurons: int
    output_neurons: int
        
    @nn.compact
    def __call__(self, x):
        z = nn.relu(nn.Dense(self.hidden_neurons)(x))
        return nn.Dense(self.output_neurons)(z)
   
MLP_singlehidden(10, 2)

MLP_singlehidden(
    # attributes
    hidden_neurons = 10
    output_neurons = 2
)

Podemos pasar argumentos al momento de construir el objeto definiendolos dentro de la clase con la notación

    nombre_variable : tipo_variable
    
:::{note}

`flax` implementa clases de tipo `dataclass` (introducidas en Python 3.7)

:::

Veamos ahora como se implementaría un modelo MLP con 

- número arbitrario de capas ocultas
- función de activación a elección (por defecto relu)

In [14]:
from typing import Sequence, Callable

class MLP(nn.Module):
    neurons_per_layer: Sequence[int]
    activation: Callable = nn.relu
    
    @nn.compact
    def __call__(self, x):
        for k, neurons in enumerate(self.neurons_per_layer):
            x = nn.Dense(neurons)(x)
            if k != len(self.hidden_neurons) - 1:
                x = self.activation(x)
        return x
    
MLP(neurons_per_layer=[10, 5, 3])

MLP(
    # attributes
    neurons_per_layer = [10, 5, 3]
    activation = relu
)

Some useful submodules of nn by category are

    Linear and Convolutional layers

    Pooling layers

    Activation functions

    Recurrent layers

    Batch normalization layers
